In [2]:
!pip install map_boxes
!pip install tqdm
!pip install pycocotools

In [1]:
from map_boxes import mean_average_precision_for_boxes
import pandas as pd
import numpy as np
import json
from tqdm import tqdm
from pycocotools.coco import COCO

In [4]:
GT_JSON = '/opt/ml/detection/dataset/train.json'
PRED_CSV = '/opt/ml/detection/faster_rcnn/faster_rcnn_torchvision_submission_2_test.csv'


    
# load ground truth
with open(GT_JSON, 'r') as outfile:
    test_anno = (json.load(outfile))

# load prediction
pred_df = pd.read_csv(PRED_CSV)

   

In [10]:
'''
[
    [file_name 1, confidence_score, x_min, x_max, y_min, y_max], 
    [file_name 2 confidence_score, x_min, x_max, y_min, y_max],
    ,,,
    [file_name , confidence_score, x_min, x_max, y_min, y_max]
]
'''
    
new_pred = []

file_names = pred_df['image_id'].values.tolist()
bboxes = pred_df['PredictionString'].values.tolist()

print(file_names[1])
print(bboxes[1])


test/0001.jpg
5 0.7475475072860718 342.19403076171875 240.119140625 769.5818481445312 708.6382446289062 7 0.46878644824028015 131.18421936035156 0.0 520.7600708007812 277.547607421875 3 0.40461161732673645 349.04498291015625 248.0780029296875 743.3067016601562 687.6563720703125 5 0.2912512719631195 134.82919311523438 4.000722408294678 502.64599609375 277.8896179199219 6 0.23765414953231812 754.1754760742188 663.9010009765625 888.736083984375 979.23193359375 0 0.16301988065242767 147.5968780517578 8.95401382446289 525.5265502929688 276.4270935058594 0 0.16107308864593506 756.2115478515625 668.2628173828125 889.0374755859375 993.9481201171875 2 0.09182024002075195 731.4864501953125 671.7282104492188 885.697265625 997.9462890625 2 0.07156708091497421 357.4524230957031 273.7116394042969 739.6365966796875 695.0551147460938 1 0.06478525698184967 757.06396484375 673.8621215820312 888.0866088867188 986.9344482421875 4 0.056180451065301895 323.448486328125 249.0672607421875 762.2232055664062 69

In [ ]:


'''
create new_pred
'''
    
for i, bbox in enumerate(bboxes):
    if isinstance(bbox, float):
        print(f'{file_names[i]} empty box')

for file_name, bbox in tqdm(zip(file_names, bboxes)):
    boxes = np.array(str(bbox).split(' '))
    
    if len(boxes) % 6 == 1:
        boxes = boxes[:-1].reshape(-1, 6)
    elif len(boxes) % 6 == 0:
        boxes = boxes.reshape(-1, 6)
    else:
        raise Exception('error', 'invalid box count')
    for box in boxes:
        new_pred.append([file_name, box[0], box[1], float(box[2]), float(box[4]), float(box[3]), float(box[5])])

In [11]:
'''
[
    [file_name 1, confidence_score, x_min, x_max, y_min, y_max], 
    [file_name 2, confidence_score, x_min, x_max, y_min, y_max],
    ,,,
    [file_name , confidence_score, x_min, x_max, y_min, y_max]
]
'''
    
gt = []

'''
create gt
'''
    
coco = COCO(GT_JSON)
   
'''
coco.getImgIds(): return image id list
    
coco.loadImgs(image_id): return image_info
    
image_info['file_name']: return file name
   
coco.getAnnIds(imgIds=image_info['id']): return annotation id
    
coco.loadAnns(ann_ids): return annotation information list (annotation_info_list)
    
annotation_info_list[i]['bbox']: return i'th annotation [x_min, y_min, w, h]
    
annotation_info_list[i]['category_id']: return i'th annotation category
    
'''
    
for image_id in coco.getImgIds():
        
    image_info = coco.loadImgs(image_id)[0]
    annotation_id = coco.getAnnIds(imgIds=image_info['id'])
    annotation_info_list = coco.loadAnns(annotation_id)
        
    file_name = image_info['file_name']
        
    for annotation in annotation_info_list:
        gt.append([file_name, annotation['category_id'],
                   float(annotation['bbox'][0]),
                   float(annotation['bbox'][0]) + float(annotation['bbox'][2]),
                   float(annotation['bbox'][1]),
                   (float(annotation['bbox'][1]) + float(annotation['bbox'][3]))])

loading annotations into memory...
Done (t=0.18s)
creating index...
index created!


In [12]:
'''
calculate mAP
'''


mean_ap, average_precisions = mean_average_precision_for_boxes(gt, new_pred, iou_threshold=0.5)

print(mean_ap)

Number of files in annotations: 4883
Number of files in predictions: 4883
Unique classes: 10
Detections length: 4883
Annotations length: 4883
0                              | 0.243367 |    3966
1                              | 0.413410 |    6352
2                              | 0.411380 |     897
3                              | 0.387853 |     936
4                              | 0.449659 |     982
5                              | 0.331874 |    2943
6                              | 0.467450 |    1263
7                              | 0.617918 |    5178
8                              | 0.547541 |     159
9                              | 0.594067 |     468
mAP: 0.446452
0.4464518988660888
